# LIBRARIES

In [31]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import requests


# DATA URL

In [32]:
link = "https://www.media.mit.edu/research/?filter=everything&tag=machine-learning"

# WEBPAGE ACCESS

In [33]:
page = requests.get(link)


# WEB SCRAPING

In [34]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content,'html.parser')

# DATA 

In [35]:
soup


<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<meta content="no" name="msapplication-tap-highlight"/>
<title>Research — MIT Media Lab</title>
<link href="/static/images/icons/apple-icon-57x57.1737f411eb73.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/static/images/icons/apple-icon-60x60.4bd2ba2d6ea4.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/static/images/icons/apple-icon-72x72.855e575da010.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/static/images/icons/apple-icon-76x76.7acd423a51c4.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/static/images/icons/apple-icon-114x114.71ce9229bafd.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/static/images/icons/apple-icon-120x120.8b383d5c8c77.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/static/images/icons/apple

# LIST DATA

In [36]:
contents  = soup.find_all('p')



# LENGTH OF DATA

In [37]:
len(contents)

223

# HTML

In [38]:
news_contents = []
list_paragraphs = []
final_article = []
for p in np.arange(0,len(contents)):
    paragraph = contents[p].get_text()
    list_paragraphs.append(paragraph)
    final_article = "".join(list_paragraphs)
news_contents.append(final_article)

# DATA CONTENT

In [39]:
News_contents= news_contents[0]
News_contents

'\n                    health\n                \n                    art\n                \n                    human-machine interaction\n                \n                    learning\n                \n                    artificial intelligence\n                \n                    robotics\n                \n                    architecture\n                \n                    design\n                \n                    technology\n                \n                    consumer electronics\n                \n                    kids\n                \n                    music\n                \n                    wearable computing\n                \n                    networks\n                \n                    politics\n                \n                    bioengineering\n                \n                    entertainment\n                \n                    cognition\n                \n                    economy\n                \n                    human-comp

# DATA CLEANING

In [40]:
News_contents=(((((News_contents.replace("\r", " ")).replace("\n", " ")).replace("    ", " ")).replace('"', '')).lower())
News_contents.replace("'s", "")
df = pd.DataFrame({News_contents})
df['Content'] = News_contents
df['Content_Parsed_1'] =(((News_contents.replace("\r", " ")).replace("\n", " ")).replace("    ", " "))
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()


# PUNCTUATIONS

In [41]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '',regex=True)

# PRONOUN

In [42]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

# STEMMING

In [43]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to C:\Users\Anand
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Anand
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# LEMMATIZATION

In [15]:
wordnet_lemmatizer = WordNetLemmatizer()

In [16]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [17]:
df['Content_Parsed_5'] = lemmatized_text_list

# STOP WORDS

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Anand
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [19]:
stop_words = list(stopwords.words('english'))

In [20]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [21]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [44]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '',regex=True)

KeyError: 'Content_Parsed_5'

In [45]:
df.head(1)

,0,Content,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4
0,health art human-mac...,health art human-mac...,health art human-machine interactio...,health art human-machine interactio...,health art human-machine interactio...,health art human-machine interactio...


In [46]:
list_columns = ["Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})


KeyError: "None of [Index(['Content_Parsed_6'], dtype='object')] are in the [columns]"

In [47]:
X_new= df['Content_Parsed']

KeyError: 'Content_Parsed'

In [48]:
loaded_model= pickle.load(open('C:\\Users\\Anand Gupta\\Desktop\\Feature Engineering\\Latest-News-Classifier-master\\0. Latest News Classifier\\04. Model Training\\Models\\best_mnbc.pickle','rb'))
X_train =pickle.load(open('C:\\Users\\Anand Gupta\\Desktop\\Feature Engineering\\Latest-News-Classifier-master\\0. Latest News Classifier\\03. Feature Engineering\\Pickles/X_train.pickle','rb'))
y_train =pickle.load(open('C:\\Users\\Anand Gupta\\Desktop\\Feature Engineering\\Latest-News-Classifier-master\\0. Latest News Classifier\\03. Feature Engineering\\Pickles/y_train.pickle','rb'))


E:\anaconda\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.19.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [49]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [50]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_new = tfidf.transform(X_new).toarray()

print(features_new.shape)

(1891, 300)
(1, 300)


In [51]:
features_new

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.07772908, 0.        , 0.07432028,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.12376281, 0.        , 0.        , 0.08457118, 0.        ,
        0.        , 0.08702294, 0.        , 0.        , 0.06796902,
        0.        , 0.        , 0.06730557, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.07863114,
        0.        , 0.        , 0.        , 0.        , 0.13829989,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.16530522, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10287356,
        0.18909757, 0.        , 0.06911578, 0.08041636, 0.        ,
        0.        , 0.08817682, 0.        , 0.        , 0.07795154,
        0.        , 0.31401102, 0.        , 0.        , 0.07863114,
        0.        , 0.        , 0.08800865, 0.  

In [52]:
loaded_model.predict(features_new)

array([4], dtype=int64)